# Brain Stroke Analysis with SQL


In [1]:
import pandas as pd

In [2]:
# importing required package
import sqlite3 as sq
import plotly.graph_objs as go
import plotly.express as px

In [3]:
# Creating Database
conn = sq.connect('brain_stroke.db')

# Create a table called brainStroke
query = """
CREATE TABLE brainStroke(
    gender VARCHAR(10),
    age DECIMAL(4,1),
    hypertension SMALLINT,
    heart_disease SMALLINT,
    ever_married VARCHAR(5),
    work_type VARCHAR(20),
    residence_type VARCHAR(10),
    avg_glucose_level DECIMAL(5,2),
    bmi DECIMAL(4,1),
    smoking_status VARCHAR(15),
    stroke SMALLINT
)
"""

In [52]:
with conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS brainStroke")
    cur.execute(query)

OperationalError: no such table: brainStroke

In [ ]:
# check our sqlite master for confirmation
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
cur.fetchall()

[('brainStroke',)]

In [ ]:
brain_df = pd.read_csv('brain_stroke.csv')

In [53]:
brain_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [61]:
csv_tuple = tuple(brain_df.itertuples(index=False, name=None))
csv_tuple[0]

('Male',
 67.0,
 0,
 1,
 'Yes',
 'Private',
 'Urban',
 228.69,
 36.6,
 'formerly smoked',
 1)

In [62]:
insert_query = "INSERT INTO brainStroke VALUES(?,?,?,?,?,?,?,?,?,?,?)"
cur.executemany(insert_query, csv_tuple)

In [18]:
query = "SELECT * FROM brainStroke LIMIT 5"
# cur.execute(query)
# cur.fetchall()

In [19]:
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


### Questions
1. What's the distribution of gender in the dataset?
2. What's the distribution of stroke in the dataset?
2. What's the distribution of brain stroke by gender.
3. For women, how many percent has stroke?
4. For men, how many percent has no stroke?
5. Details of the oldest people with stroke
6. How many people are married and has stroke?
7. How many people are not married and has stroke?
7. Categorize bmi into three group after checking the max, min, and avg.
8. Categorize avg_glucose_level into three group after checking max, min and avg

1

In [20]:
query = """
SELECT    gender, COUNT(*) count
FROM      brainStroke
GROUP BY  gender
"""
pd.read_sql_query(query, conn)

,gender,count
0,Female,2907
1,Male,2074


2

In [21]:
query = """
SELECT    stroke, COUNT(*) count
FROM      brainStroke
GROUP BY  stroke
"""
pd.read_sql_query(query, conn)

,stroke,count
0,0,4733
1,1,248


3

In [58]:
query = """
SELECT    gender, stroke, COUNT(*) count
FROM      brainStroke
GROUP BY  gender, stroke
"""
pd.read_sql_query(query, conn)

,gender,stroke,count
0,Female,0,2767
1,Female,1,140
2,Male,0,1966
3,Male,1,108


In [65]:
query = """
SELECT      stroke, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE gender = 'Female') total
WHERE       gender = 'Female'
GROUP BY    stroke

UNION ALL

SELECT      stroke, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE gender = 'Male') total
WHERE       gender = 'Male'
GROUP BY    stroke
"""
pd.read_sql_query(query, conn)

,stroke,Percent
0,0,95.184039
1,1,4.815961
2,0,94.792671
3,1,5.207329


In [66]:
query = """
SELECT * 
FROM    brainStroke
WHERE   age > 70
"""
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
1,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
2,Male,81,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
3,Male,74,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
4,Female,78,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...
702,Female,82,1,0,Yes,Private,Urban,222.52,31.8,formerly smoked,0
703,Female,78,0,0,Yes,Govt_job,Urban,101.76,27.3,smokes,0
704,Female,78,1,1,Yes,Private,Rural,206.53,31.2,never smoked,0
705,Male,72,0,1,Yes,Private,Rural,238.27,30.7,smokes,0


In [52]:
query = """
  SELECT gender, COUNT(*) has_both_hyptnsn_hrt_dis
    FROM brainStroke
   WHERE hypertension = 1 AND heart_disease = 1
GROUP BY gender
"""
pd.read_sql_query(query, conn)

,gender,has_both_hyptnsn_hrt_dis
0,Female,29
1,Male,35


In [56]:
query = """
  SELECT residence_type, stroke,  COUNT(*) count
    FROM brainStroke 
GROUP BY residence_type, stroke
"""
pd.read_sql_query(query, conn)

,residence_type,stroke,count
0,Rural,0,2336
1,Rural,1,113
2,Urban,0,2397
3,Urban,1,135


In [57]:
query = """
SELECT MAX(age) max_age, MIN(age) min_age, AVG(age) avg_age
  FROM brainStroke
"""
pd.read_sql_query(query, conn)

,max_age,min_age,avg_age
0,82,0.08,43.419859


In [62]:
query = """
SELECT *
  FROM brainStroke
 WHERE age < 10
 LIMIT 10
"""
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,3.00,0,0,No,children,Rural,95.12,18.0,Unknown,0
1,Female,8.00,0,0,No,Private,Urban,110.89,17.6,Unknown,0
2,Female,3.00,0,0,No,children,Urban,73.74,16.0,Unknown,0
3,Male,4.00,0,0,No,children,Rural,79.17,20.0,Unknown,0
4,Male,8.00,0,0,No,children,Rural,78.05,25.7,Unknown,0
5,Female,0.64,0,0,No,children,Urban,83.82,24.9,Unknown,0
6,Female,4.00,0,0,No,children,Rural,93.25,16.6,Unknown,0
7,Female,0.88,0,0,No,children,Rural,88.11,15.5,Unknown,0
8,Male,4.00,0,0,No,children,Rural,106.22,16.7,Unknown,0
9,Male,5.00,0,0,No,children,Rural,85.84,16.4,Unknown,0


In [ ]:
query = """
SELECT MAX(age) max_age, MIN(age) min_age, AVG(age) avg_age
FROM brainStroke
"""
pd.read_sql_query(query, conn)

In [61]:
query = """
SELECT DISTINCT(work_type)
FROM brainStroke
"""
pd.read_sql_query(query, conn)

,work_type
0,Private
1,Self-employed
2,Govt_job
3,children


In [63]:
query = """
SELECT DISTINCT(smoking_status)
FROM brainStroke
"""
pd.read_sql_query(query, conn)

,smoking_status
0,formerly smoked
1,never smoked
2,smokes
3,Unknown
